In [1]:
# !pip install -U torchdata
# !pip install -U spacy
# !python -m spacy download en_core_web_lg
# !python -m spacy download de_core_news_lg
# !pip install portalocker
! pip install accelerate -U
! pip install evaluate
! pip install sentence-transformers
! pip install SentencePiece
! pip install bert_score
! pip install --upgrade nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 38.7 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.


In [2]:
#  import T5
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
import numpy as np
import os

import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    DebertaTokenizer,
    DebertaForSequenceClassification,
    AutoConfig,
)

import logging
import evaluate
from evaluate import load
from datasets import load_dataset

from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import T5Tokenizer,T5ForConditionalGeneration

import nltk
from nltk.translate.bleu_score import corpus_bleu

import matplotlib.pyplot as plt

from  transformers  import  AutoTokenizer, AutoModelWithLMHead

import csv
from torch.utils.data import DataLoader, Dataset
import json
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pickle
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
torch.manual_seed(1212)
device=""
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'Using GPU: {torch.cuda.get_device_name()}')
else:
    device = torch.device("cpu")
    print('Using CPU')

2024-04-24 10:18:28.848742: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 10:18:28.848842: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 10:18:28.992044: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using GPU: Tesla P100-PCIE-16GB


In [3]:
PATH="/kaggle/input/webis-clickbait-22/"
OUTPATH="/kaggle/working/"
# PATH="D:\\ghd\\NLP-Project\\webis-clickbait-22\\"
# OUTPATH="D:\\ghd\\NLP-Project\\webis-clickbait-22\\output\\"
BATCH=8
T5_model = "t5-base"
sep='[SEP]'

In [4]:
# data=pd.read_json(PATH+"train.jsonl",lines=True,encoding='utf-8')
# data.head()

In [5]:
def get_split_data(file_path,if_train=True):
    global label_encoder
    data=pd.read_json(file_path,lines=True,encoding='utf-8')

    # y = data["spoiler"]
    Y=[]
    # for i in y:
    #     Y.append(" ".join(i))

    new_data=data[['postText','targetParagraphs','targetTitle','targetDescription','targetKeywords','spoiler','tags']]
    dic1=new_data.to_dict(orient='records')

    dic=[]
    for i in dic1:
        # print(i)
        if('phrase' not in i['tags']):
            del i['postText']
            del i['targetParagraphs']
            del i['targetTitle']
            del i['targetDescription']
            del i['targetKeywords']
            del i['tags']
            del i['spoiler']
            del i
            continue

        i['postText']=i['postText'][0]
        targetparah=" ".join(i['targetParagraphs'])
        i['targetParagraphs']=targetparah
        j={}
        j['question']="question : "+i['postText']+"?"
        j['context']="context : "+i['targetParagraphs']
        dic.append(j)
        Y.append(" ".join(i['spoiler']))

        del i['postText']
        del i['targetParagraphs']
        del i['targetTitle']
        del i['targetDescription']
        del i['targetKeywords']
        del i['tags']
        del i['spoiler']
        # break
    # print("desccount: ",desccount)
    # print("keycount: ",keycount)
#     print(dic)
    X_q = [i['question'] for i in dic]
    X_q=np.array(X_q)
    X_c = [i['context'] for i in dic]
    X_c=np.array(X_c)

    # return X_q[:100],X_c[:100],Y[:100]
    return X_q,X_c,Y



In [6]:
x_train_q,x_train_c,y_train = get_split_data(PATH+'train.jsonl')
x_test_q,x_test_c,y_test = get_split_data(PATH+'test.jsonl',False)
x_val_q,x_val_c,y_val = get_split_data(PATH+'validation.jsonl',False)

In [7]:
x_train_q.shape,x_train_c.shape,len(y_train),x_test_q.shape,x_test_c.shape,len(y_test),x_val_q.shape,x_val_c.shape,len(y_val)

((1367,), (1367,), 1367, (423,), (423,), 423, (335,), (335,), 335)

In [8]:
# create pd dataframe
train_df = pd.DataFrame(list(zip(x_train_q, x_train_c, y_train)), columns =['question', 'context', 'spoiler'])
val_df = pd.DataFrame(list(zip(x_val_q, x_val_c, y_val)), columns =['question', 'context', 'spoiler'])
test_df = pd.DataFrame(list(zip(x_test_q, x_test_c, y_test)), columns =['question', 'context', 'spoiler'])


In [9]:
train_df["question"][0],train_df["context"][0],train_df["spoiler"][0]

('question : NASA sets date for full recovery of ozone hole?',
 'context : 2070 is shaping up to be a great year for Mother Earth. That\'s when NASA scientists are predicting the hole in the ozone layer might finally make a full recovery. Researchers announced their conclusion, in addition to other findings, in a presentation Wednesday during the annual American Geophysical Union meeting in San Francisco. The team of scientists specifically looked at the chemical composition of the ozone hole, which has shifted in both size and depth since the passing of the Montreal Protocol in 1987. The agreement banned its 197 signatory countries from using chemicals, like chlorofluorocarbons (CFCs), that break down into chlorine in the upper atmosphere and harm the ozone layer. They found that, while levels of chlorine in the atmosphere have indeed decreased as a result of the protocol, it\'s too soon to tie them to a healthier ozone layer. "Ozone holes with smaller areas and a larger total amount 

In [10]:
# t5
tokenizer = AutoTokenizer.from_pretrained(T5_model)
model = AutoModelWithLMHead.from_pretrained(T5_model)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:1682: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [11]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

### convert to Huggingface dataset
train_data = Dataset(pa.Table.from_pandas(train_df))
val_data = Dataset(pa.Table.from_pandas(val_df))
test_data = Dataset(pa.Table.from_pandas(test_df))
train_data,val_data,test_data

(Dataset({
     features: ['question', 'context', 'spoiler'],
     num_rows: 1367
 }),
 Dataset({
     features: ['question', 'context', 'spoiler'],
     num_rows: 335
 }),
 Dataset({
     features: ['question', 'context', 'spoiler'],
     num_rows: 423
 }))

In [12]:

train_data[0]

{'question': 'question : NASA sets date for full recovery of ozone hole?',
 'context': 'context : 2070 is shaping up to be a great year for Mother Earth. That\'s when NASA scientists are predicting the hole in the ozone layer might finally make a full recovery. Researchers announced their conclusion, in addition to other findings, in a presentation Wednesday during the annual American Geophysical Union meeting in San Francisco. The team of scientists specifically looked at the chemical composition of the ozone hole, which has shifted in both size and depth since the passing of the Montreal Protocol in 1987. The agreement banned its 197 signatory countries from using chemicals, like chlorofluorocarbons (CFCs), that break down into chlorine in the upper atmosphere and harm the ozone layer. They found that, while levels of chlorine in the atmosphere have indeed decreased as a result of the protocol, it\'s too soon to tie them to a healthier ozone layer. "Ozone holes with smaller areas and

In [13]:
def preproc(data):
    q=[]
    c=[]
    a=[]
    # print(len(data))
    # print(len(data['question']))
    # print(len(data['context']))
    # print(len(data['spoiler']))
    for i in range(len(data['question'])):
        q.append(data['question'][i])
        c.append(data['context'][i])
        a.append(data['spoiler'][i])

    model_inputs=tokenizer(q,c,text_target=a,return_tensors='pt',padding=True,truncation=True,max_length=512)
    return model_inputs


In [14]:
# train_data=Dataset.from_pandas(train_df)
tokenized_train=train_data.map(preproc,batched=True,batch_size=BATCH)
tokenized_val=val_data.map(preproc,batched=True,batch_size=BATCH)
tokenized_test=test_data.map(preproc,batched=True,batch_size=BATCH)
tokenized_train,tokenized_val,tokenized_test

Map:   0%|          | 0/1367 [00:00<?, ? examples/s]

Map:   0%|          | 0/335 [00:00<?, ? examples/s]

Map:   0%|          | 0/423 [00:00<?, ? examples/s]

(Dataset({
     features: ['question', 'context', 'spoiler', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 1367
 }),
 Dataset({
     features: ['question', 'context', 'spoiler', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 335
 }),
 Dataset({
     features: ['question', 'context', 'spoiler', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 423
 }))

In [15]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=T5_model)

In [16]:
import wandb
api_key = "9963cf6219e451d47251ea34645181ada1b2526b"
wandb.login(key=api_key)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [17]:
model = AutoModelForSeq2SeqLM.from_pretrained(T5_model)
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [18]:
bertscore = load("bertscore")
meteor = evaluate.load("meteor")
bleu = evaluate.load("bleu")

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [19]:

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    pred, ref = postprocess_text(decoded_preds, decoded_labels)

    # cal bleu, meteor and bert
    bleu_score = bleu.compute(predictions=pred, references=ref)
#     bleu1 = corpus_bleu(ref,pred,weights=(1,0,0,0))
#     bleu2 = corpus_bleu(ref,pred,weights=(0.5,0.5,0))
#     bleu3 = corpus_bleu(ref,pred,weights=(0.33,0.33,0.33,0)
#     bleu4 = corpus_bleu(ref,pred,weights=(0.25,0.25,0.25,0.25))

    print(bleu_score)
    meteor_score = meteor.compute(predictions=pred, references=ref)
    bertscore_score = bertscore.compute(predictions=pred, references=ref, lang="en")

    #  dict
    return {"blue":bleu_score["bleu"],
            "precisions_1":bleu_score["precisions"][0],
            "precisions_2":bleu_score["precisions"][1],
            "precisions_3":bleu_score["precisions"][2],
            "precisions_4":bleu_score["precisions"][3],
            "bp":bleu_score["brevity_penalty"],
            "meteor": meteor_score["meteor"], 
            "bertscore_f1": np.average(bertscore_score["f1"]), 
            "bertscore_p": np.average(bertscore_score["precision"]), 
            "bertscore_r": np.average(bertscore_score["recall"])}


In [20]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./T5",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH,
    per_device_eval_batch_size=BATCH,
    weight_decay=0.01,
    save_total_limit=4,
    num_train_epochs=12,
    predict_with_generate=True,
    save_strategy="epoch",
    fp16=True,
#     report_to="wandb",
#     logging_dir='./lolololol'

    # push_to_hub=True
    # load_best_model_at_end=True,
    # metric_for_best_model="bleu"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Currently logged in as: raghav21274 (ragha). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240424_101857-ue49gvj7
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run rich-valley-51
wandb: ⭐️ View project at https://wandb.ai/ragha/huggingface
wandb: 🚀 View run at https://wandb.ai/ragha/huggingface/runs/ue49gvj7


Epoch,Training Loss,Validation Loss,Blue,Precisions 1,Precisions 2,Precisions 3,Precisions 4,Bp,Meteor,Bertscore F1,Bertscore P,Bertscore R
1,No log,0.472713,0.273746,0.496560,0.375926,0.227425,0.132275,1.000000,0.465423,0.916541,0.918290,0.915940
2,No log,0.420801,0.329165,0.558685,0.435203,0.278986,0.174699,0.997655,0.505501,0.931562,0.932973,0.931072
3,1.187100,0.409459,0.348582,0.572581,0.452158,0.298969,0.190751,1.000000,0.524071,0.934212,0.935398,0.933976
4,1.187100,0.405401,0.370516,0.573481,0.454386,0.318885,0.226804,1.000000,0.534448,0.936297,0.936549,0.936896
5,1.187100,0.406555,0.354232,0.564744,0.441781,0.299401,0.210784,1.000000,0.539180,0.936282,0.935816,0.937597
6,0.344000,0.404636,0.360385,0.569414,0.448041,0.306548,0.215686,1.000000,0.543404,0.937213,0.937252,0.938061
7,0.344000,0.412602,0.373444,0.581036,0.459790,0.320988,0.226804,1.000000,0.544263,0.938121,0.938330,0.938781
8,0.344000,0.414764,0.386451,0.593857,0.468750,0.332215,0.241176,1.000000,0.540567,0.937948,0.938288,0.938444
9,0.255100,0.421079,0.390312,0.587640,0.466667,0.336601,0.251429,1.000000,0.540249,0.936783,0.937162,0.937254
10,0.255100,0.425634,0.398060,0.592299,0.472628,0.344482,0.260355,1.000000,0.536908,0.936629,0.937429,0.936685


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'bleu': 0.2737457621485182, 'precisions': [0.49655963302752293, 0.37592592592592594, 0.22742474916387959, 0.13227513227513227], 'brevity_penalty': 1.0, 'length_ratio': 1.0210772833723654, 'translation_length': 872, 'reference_length': 854}


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'bleu': 0.3928302912903953, 'precisions': [0.5898876404494382, 0.46846846846846846, 0.33881578947368424, 0.2543352601156069], 'brevity_penalty': 1.0, 'length_ratio': 1.0421545667447307, 'translation_length': 890, 'reference_length': 854}


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'bleu': 0.38868804639781324, 'precisions': [0.590293453724605, 0.4664246823956443, 0.33554817275747506, 0.24705882352941178], 'brevity_penalty': 1.0, 'length_ratio': 1.0374707259953162, 'translation_length': 886, 'reference_length': 854}


TrainOutput(global_step=2052, training_loss=0.49181168493006894, metrics={'train_runtime': 1656.4574, 'train_samples_per_second': 9.903, 'train_steps_per_second': 1.239, 'total_flos': 9989345236746240.0, 'train_loss': 0.49181168493006894, 'epoch': 12.0})

In [21]:
trainer.save_model("T5_overall")

In [22]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [23]:
results = trainer.evaluate(eval_dataset=tokenized_test)
print(results)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'bleu': 0.4615501144354541, 'precisions': [0.6025751072961374, 0.5175202156334232, 0.41849148418491483, 0.35864978902953587], 'brevity_penalty': 0.9923044417169341, 'length_ratio': 0.9923339011925043, 'translation_length': 1165, 'reference_length': 1174}
{'eval_loss': 0.49083593487739563, 'eval_blue': 0.4615501144354541, 'eval_precisions_1': 0.6025751072961374, 'eval_precisions_2': 0.5175202156334232, 'eval_precisions_3': 0.41849148418491483, 'eval_precisions_4': 0.35864978902953587, 'eval_bp': 0.9923044417169341, 'eval_meteor': 0.5674285262128053, 'eval_bertscore_f1': 0.9350105405699277, 'eval_bertscore_p': 0.9364802096188773, 'eval_bertscore_r': 0.9342610345664599, 'eval_runtime': 31.8017, 'eval_samples_per_second': 13.301, 'eval_steps_per_second': 1.667, 'epoch': 12.0}


In [24]:
def calculate_bleu(bp,precisions):
    weights_1 = np.array([1,0,0,0])
    weights_2 = np.array([0.5,0.5,0,0])
    weights_3 = np.array([0.33,0.33,0.33,0])
    weights_4 = np.array([0.25,0.25,0.25,0.25])
    logp = np.log(precisions)
        
    res1 = bp*np.exp(np.dot(logp,weights_1))
    res2 = bp*np.exp(np.dot(logp,weights_2))
    res3 = bp*np.exp(np.dot(logp,weights_3))
    res4 = bp*np.exp(np.dot(logp,weights_4))
    
    return [res1,res2,res3,res4]

In [25]:
results = trainer.evaluate(eval_dataset=tokenized_train)
print(results)
calculate_bleu(results['eval_bp'],[results['eval_precisions_1'],results['eval_precisions_2'],results['eval_precisions_3'],results['eval_precisions_4']])

{'bleu': 0.7670663952253861, 'precisions': [0.84688995215311, 0.8133611691022965, 0.7626728110599078, 0.7039030955585465], 'brevity_penalty': 0.983654466599438, 'length_ratio': 0.983786610878661, 'translation_length': 3762, 'reference_length': 3824}
{'eval_loss': 0.1475869119167328, 'eval_blue': 0.7670663952253861, 'eval_precisions_1': 0.84688995215311, 'eval_precisions_2': 0.8133611691022965, 'eval_precisions_3': 0.7626728110599078, 'eval_precisions_4': 0.7039030955585465, 'eval_bp': 0.983654466599438, 'eval_meteor': 0.7586530951633871, 'eval_bertscore_f1': 0.9767891172518587, 'eval_bertscore_p': 0.9775543943232834, 'eval_bertscore_r': 0.9763058644188698, 'eval_runtime': 101.4018, 'eval_samples_per_second': 13.481, 'eval_steps_per_second': 1.686, 'epoch': 12.0}


[0.833047084153591, 0.8163901870140239, 0.7954092414410536, 0.7670663952253861]

In [26]:
results = trainer.evaluate(eval_dataset=tokenized_val)
print(results)
calculate_bleu(results['eval_bp'],[results['eval_precisions_1'],results['eval_precisions_2'],results['eval_precisions_3'],results['eval_precisions_4']])

{'bleu': 0.38868804639781324, 'precisions': [0.590293453724605, 0.4664246823956443, 0.33554817275747506, 0.24705882352941178], 'brevity_penalty': 1.0, 'length_ratio': 1.0374707259953162, 'translation_length': 886, 'reference_length': 854}
{'eval_loss': 0.43035054206848145, 'eval_blue': 0.38868804639781324, 'eval_precisions_1': 0.590293453724605, 'eval_precisions_2': 0.4664246823956443, 'eval_precisions_3': 0.33554817275747506, 'eval_precisions_4': 0.24705882352941178, 'eval_bp': 1.0, 'eval_meteor': 0.5384067506004799, 'eval_bertscore_f1': 0.9363934027614879, 'eval_bertscore_p': 0.9369333343719368, 'eval_bertscore_r': 0.9367215692107357, 'eval_runtime': 25.4287, 'eval_samples_per_second': 13.174, 'eval_steps_per_second': 1.652, 'epoch': 12.0}


[0.590293453724605,
 0.5247165298270361,
 0.4556689561939599,
 0.38868804639781324]

In [27]:
results = trainer.evaluate(eval_dataset=tokenized_test)
print(results)
calculate_bleu(results['eval_bp'],[results['eval_precisions_1'],results['eval_precisions_2'],results['eval_precisions_3'],results['eval_precisions_4']])

{'bleu': 0.4615501144354541, 'precisions': [0.6025751072961374, 0.5175202156334232, 0.41849148418491483, 0.35864978902953587], 'brevity_penalty': 0.9923044417169341, 'length_ratio': 0.9923339011925043, 'translation_length': 1165, 'reference_length': 1174}
{'eval_loss': 0.49083593487739563, 'eval_blue': 0.4615501144354541, 'eval_precisions_1': 0.6025751072961374, 'eval_precisions_2': 0.5175202156334232, 'eval_precisions_3': 0.41849148418491483, 'eval_precisions_4': 0.35864978902953587, 'eval_bp': 0.9923044417169341, 'eval_meteor': 0.5674285262128053, 'eval_bertscore_f1': 0.9350105405699277, 'eval_bertscore_p': 0.9364802096188773, 'eval_bertscore_r': 0.9342610345664599, 'eval_runtime': 31.7331, 'eval_samples_per_second': 13.33, 'eval_steps_per_second': 1.67, 'epoch': 12.0}


[0.5979379554380153, 0.554133222030151, 0.5067587176869219, 0.4615501144354541]

In [28]:
q='question : Wes Welker Wanted Dinner With Tom Brady, But Patriots QB Had Better Idea?'
c='context : It’ll be just like old times this weekend for Tom Brady and Wes Welker. Welker revealed Friday morning on a Miami radio station that he contacted Brady because he’ll be in town for Sunday’s game between the New England Patriots and Miami Dolphins at Gillette Stadium. It seemed like a perfect opportunity for the two to catch up. But Brady’s definition of "catching up" involves far more than just a meal. In fact, it involves some literal "catching" as the Patriots quarterback looks to stay sharp during his four-game Deflategate suspension. "I hit him up to do dinner Saturday night. He’s like, ‘I’m going to be flying in from Ann Arbor later (after the Michigan-Colorado football game), but how about that morning we go throw?’ " Welker said on WQAM, per The Boston Globe. "And I’m just sitting there, I’m like, ‘I was just thinking about dinner, but yeah, sure. I’ll get over there early and we can throw a little bit.’ " Welker was one of Brady’s favorite targets for six seasons from 2007 to 2012. It’s understandable him and Brady want to meet with both being in the same area. But Brady typically is all business during football season. Welker probably should have known what he was getting into when reaching out to his buddy. "That’s the only thing we really have planned," Welker said of his upcoming workout with Brady. "It’s just funny. I’m sitting there trying to have dinner. ‘Hey, get your ass up here and let’s go throw.’ I’m like, ‘Aw jeez, man.’ He’s going to have me running like 2-minute drills in his backyard or something." Maybe Brady will put a good word in for Welker down in Foxboro if the former Patriots wide receiver impresses him enough.'
enc=tokenizer(q+" "+c,return_tensors='pt',padding=True,truncation=True,max_length=512)
out=model.generate(input_ids=enc['input_ids'].to(device))

In [29]:
tokenizer.decode(out[0]),'how about that morning we go throw?'

('<pad> I’ll get over there early and we can throw a little bit</s>',
 'how about that morning we go throw?')

In [30]:
import os
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/working/__notebook__.ipynb
/kaggle/working/T5/checkpoint-2052/config.json
/kaggle/working/T5/checkpoint-2052/model.safetensors
/kaggle/working/T5/checkpoint-2052/generation_config.json
/kaggle/working/T5/checkpoint-2052/scheduler.pt
/kaggle/working/T5/checkpoint-2052/special_tokens_map.json
/kaggle/working/T5/checkpoint-2052/tokenizer_config.json
/kaggle/working/T5/checkpoint-2052/optimizer.pt
/kaggle/working/T5/checkpoint-2052/trainer_state.json
/kaggle/working/T5/checkpoint-2052/spiece.model
/kaggle/working/T5/checkpoint-2052/tokenizer.json
/kaggle/working/T5/checkpoint-2052/training_args.bin
/kaggle/working/T5/checkpoint-2052/rng_state.pth
/kaggle/working/T5/checkpoint-1539/config.json
/kaggle/working/T5/checkpoint-1539/model.safetensors
/kaggle/working/T5/checkpoint-1539/generation_config.json
/kaggle/working/T5/checkpoint-1539/scheduler.pt
/kaggle/working/T5/checkpoint-1539/special_tokens_map.json
/kaggle/working/T5/checkpoint-1539/tokenizer_config.json
/kaggle/working/T5/